<a href="https://colab.research.google.com/github/tsong44/BodoSugesstions/blob/main/NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1)

In [93]:
# Read the dataset
df = pd.read_csv('/content/drive/MyDrive/Datasets/modified_dataset.csv', encoding='utf-8')
# Reorder the columns
df = df[['english', 'hindi']]

In [94]:
# Add a new column filled with "ted" at the beginning
df.insert(0, 'source', 'ted')

In [95]:
# Reorder the columns if needed (optional, since insert already places it first)
df = df[['source', 'english', 'hindi']]

In [96]:
# Take first 1000 rows for better vocabulary coverage
df.head(100)

,source,english,hindi
0,ted,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1,ted,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक
2,ted,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लगइन खाका
3,ted,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका
4,ted,a list of plugins that are disabled by default,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है
5,ted,highlight duration,अवधि को हाइलाइट रकें
6,ted,the duration of the highlight box when selecting accessible nodes,पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट बक्से की अवधि
7,ted,highlight border color,सीमांत बोर्डर के रंग को हाइलाइट करें
8,ted,the color and opacity of the highlight border,हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता।
9,ted,highlight fill color,भराई के रंग को हाइलाइट करें


In [97]:
pd.isnull(df).sum()

,0
source,0
english,0
hindi,0


In [98]:
df.shape

(30000, 3)

In [99]:
# Add start and end tokens to target sequences
df['hindi'] = df['hindi'].apply(lambda x : 'START_ '+ x + ' _END')

In [100]:
df.head()

,source,english,hindi
0,ted,give your application an accessibility workout,START_ अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें _END
1,ted,accerciser accessibility explorer,START_ एक्सेर्साइसर पहुंचनीयता अन्वेषक _END
2,ted,the default plugin layout for the bottom panel,START_ निचले पटल के लिए डिफोल्ट प्लगइन खाका _END
3,ted,the default plugin layout for the top panel,START_ ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका _END
4,ted,a list of plugins that are disabled by default,START_ उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है _END


In [101]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in df['english']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in df['hindi']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [102]:
len(all_eng_words)

7981

In [103]:
len(all_hindi_words)

9934

In [104]:
df['length_eng_sentence']=df['english'].apply(lambda x:len(x.split(" ")))
df['length_hin_sentence']=df['hindi'].apply(lambda x:len(x.split(" ")))

In [105]:
df.head(50)

,source,english,hindi,length_eng_sentence,length_hin_sentence
0,ted,give your application an accessibility workout,START_ अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें _END,6,10
1,ted,accerciser accessibility explorer,START_ एक्सेर्साइसर पहुंचनीयता अन्वेषक _END,3,5
2,ted,the default plugin layout for the bottom panel,START_ निचले पटल के लिए डिफोल्ट प्लगइन खाका _END,8,9
3,ted,the default plugin layout for the top panel,START_ ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका _END,8,9
4,ted,a list of plugins that are disabled by default,START_ उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है _END,9,14
5,ted,highlight duration,START_ अवधि को हाइलाइट रकें _END,2,6
6,ted,the duration of the highlight box when selecting accessible nodes,START_ पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट बक्से की अवधि _END,10,12
7,ted,highlight border color,START_ सीमांत बोर्डर के रंग को हाइलाइट करें _END,3,9
8,ted,the color and opacity of the highlight border,START_ हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। _END,8,10
9,ted,highlight fill color,START_ भराई के रंग को हाइलाइट करें _END,3,8


In [106]:
df[df['length_eng_sentence']>30].shape

(200, 5)

In [107]:
df=df[df['length_eng_sentence']<=20]
df=df[df['length_hin_sentence']<=20]

In [108]:
df.shape

(29104, 5)

In [109]:
print("maximum length of Hindi Sentence ",max(df['length_hin_sentence']))
print("maximum length of English Sentence ",max(df['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [110]:
max_length_src=max(df['length_hin_sentence'])
max_length_tar=max(df['length_eng_sentence'])

In [111]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(7981, 9934)

In [112]:
num_decoder_tokens = num_decoder_tokens + 1 #for zero padding
num_encoder_tokens = num_encoder_tokens + 1

In [113]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [114]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [115]:
df = shuffle(df)
df.head(10)

,source,english,hindi,length_eng_sentence,length_hin_sentence
14774,ted,t ype,START_ प्रकारः _END,2,3
27282,ted,you give channel owner privileges to yourself,START_ आपने चैनल मालिक का विशेषाधिकार अपने आप को दिया है _END,7,12
1320,ted,step over,START_ सर्वर सरवर _END,2,4
5124,ted,blank,START_ खाली _END,1,3
24044,ted,remove the current selected value from the list,START_ इस सूची में से मौजूदा चयनिद मूल्य को मिटाएँ _END,8,11
26177,ted,muted trumpet,START_ म्यूटेड तुरही _END,2,4
5387,ted,use an external md file that stores the checksum of a disc,START_ किसी बाहरी का प्रयोग करें फ़ाइल जो डिस्क के को जमा करता है _END,12,15
24655,ted,loading plug ins,START_ प्लगइन लोड किया जा रहा है _END,3,8
5990,ted,mouse button to use in the scanning mode,START_ स्कैनिंग विधि में उपयोग के लिए माउस बटन _END,8,10
7407,ted,rename group,START_ समूह का फिर नाम दें _END,2,7


In [116]:
X, y = df['english'], df['hindi']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((23283,), (5821,))

In [117]:
X_train.to_pickle('/content/drive/MyDrive/Datasets/PKL/X_train.pkl')
X_test.to_pickle('/content/drive/MyDrive/Datasets/PKL/X_test.pkl')

In [118]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src), dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar), dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens), dtype='float32')

            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    if t < max_length_src:
                        encoder_input_data[i, t] = input_token_index[word]

                for t, word in enumerate(target_text.split()):
                    if t < max_length_tar-1:
                        decoder_input_data[i, t] = target_token_index[word]
                    if t > 0 and t < max_length_tar:
                        decoder_target_data[i, t-1, target_token_index[word]] = 1.

            yield {"encoder_inputs": encoder_input_data,
                  "decoder_inputs": decoder_input_data}, decoder_target_data

In [119]:
# Define training parameters
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 10

In [120]:
latent_dim=300

In [121]:
# First, define input layers with specific names
encoder_inputs = Input(shape=(None,), name='encoder_inputs')
decoder_inputs = Input(shape=(None,), name='decoder_inputs')

In [122]:
# Encoder
enc_emb = Embedding(num_encoder_tokens, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [123]:
# Decoder
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [124]:
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [125]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [126]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_6 (Embedding)   │ (None, None, 300)      │      2,394,600 │ encoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_4 (NotEqual)    │ (None, None)           │              0 │ encoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_7 (Embedding)   │ (None, None, 300)      │      2,980,500 │ decoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_6 (LSTM)             │ [(None, 300), (None,   │        721,200 │ embedding_6[0][0],     │
│                           │ 300), (None, 300)]     │                │ not_equal_4[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_7 (LSTM)             │ [(None, None, 300),    │        721,200 │ embedding_7[0][0],     │
│                           │ (None, 300), (None,    │                │ lstm_6[0][1],          │
│                           │ 300)]                  │                │ lstm_6[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, None, 9935)     │      2,990,435 │ lstm_7[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 9,807,935 (37.41 MB)

 Trainable params: 9,807,935 (37.41 MB)

 Non-trainable params: 0 (0.00 B)

In [127]:
import tensorflow as tf

In [128]:
# Create tf.data.Dataset objects for training and validation
train_dataset = tf.data.Dataset.from_generator(
    lambda: generate_batch(X_train, y_train, batch_size),
    output_signature=(
        {
            "encoder_inputs": tf.TensorSpec(shape=(batch_size, max_length_src), dtype=tf.float32),
            "decoder_inputs": tf.TensorSpec(shape=(batch_size, max_length_tar), dtype=tf.float32)
        },
        tf.TensorSpec(shape=(batch_size, max_length_tar, num_decoder_tokens), dtype=tf.float32)
    )
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: generate_batch(X_test, y_test, batch_size),
    output_signature=(
        {
            "encoder_inputs": tf.TensorSpec(shape=(batch_size, max_length_src), dtype=tf.float32),
            "decoder_inputs": tf.TensorSpec(shape=(batch_size, max_length_tar), dtype=tf.float32)
        },
        tf.TensorSpec(shape=(batch_size, max_length_tar, num_decoder_tokens), dtype=tf.float32)
    )
)


In [129]:
# Train the model
model.fit(
    train_dataset,
    steps_per_epoch=train_samples//batch_size,
    epochs=epochs,
    validation_data=val_dataset,
    validation_steps=val_samples//batch_size
)

Epoch 1/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 1037s 6s/step - loss: 6.3190 - val_loss: 5.0612
Epoch 2/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 1041s 6s/step - loss: 5.0126 - val_loss: 4.9264
Epoch 3/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 1061s 6s/step - loss: 4.8828 - val_loss: 4.8498
Epoch 4/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 1039s 6s/step - loss: 4.8060 - val_loss: 4.7968
Epoch 5/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 1035s 6s/step - loss: 4.7423 - val_loss: 4.7353
Epoch 6/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 1062s 6s/step - loss: 4.6684 - val_loss: 4.6687
Epoch 7/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 1075s 6s/step - loss: 4.5916 - val_loss: 4.5877
Epoch 8/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 981s 5s/step - loss: 4.4976 - val_loss: 4.5134
Epoch 9/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 982s 5s/step - loss: 4.4010 - val_loss: 4.4286
Epoch 10/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 1040s 6s/step - loss: 4.3051 - val_loss: 4.3435


In [131]:
model.save_weights('/content/drive/MyDrive/Datasets/PKL/nmt.weights.h5')

In [134]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [135]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [136]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [144]:
k += 1
input_seq_dict, actual_output = next(train_gen)  # Get the next batch of data

# Access the 'encoder_inputs' key from the dictionary
input_seq = input_seq_dict['encoder_inputs']

decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k + 1].values[0])
print('Actual Hindi Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 785ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Input English sentence: are you sure you want to permanently remove the recipe
Actual Hindi Translation:  क्या वाकई आप रेसिपि को हमेशा के लिए मिटाना चाहते हैं 
Predicted Hindi Translation:  क्या आप निश्चित है कि आप आप आप चाहते हैं 


In [145]:
k += 1
input_seq_dict, actual_output = next(train_gen)  # Get the next batch of data

# Access the 'encoder_inputs' key from the dictionary
input_seq = input_seq_dict['encoder_inputs']

decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k + 1].values[0])
print('Actual Hindi Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Input English sentence: failing to open database trying to create it
Actual Hindi Translation:  डाटाबेस खोलने में असफल इसे बनाने की कोशिश की जा रही है 
Predicted Hindi Translation:  फ़ाइल के लिए संदेश में त्रुटि में त्रुटि 


In [146]:
k += 1
input_seq_dict, actual_output = next(train_gen)  # Get the next batch of data

# Access the 'encoder_inputs' key from the dictionary
input_seq = input_seq_dict['encoder_inputs']

decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k + 1].values[0])
print('Actual Hindi Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Input English sentence: the contact is offline
Actual Hindi Translation:  संपर्क ऑफ़लाइन हो गया 
Predicted Hindi Translation:  कोई कोई नहीं है 


In [147]:
k += 1
input_seq_dict, actual_output = next(train_gen)  # Get the next batch of data

# Access the 'encoder_inputs' key from the dictionary
input_seq = input_seq_dict['encoder_inputs']

decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k + 1].values[0])
print('Actual Hindi Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
Input English sentence: quit now
Actual Hindi Translation:  बाहर 
Predicted Hindi Translation:  अगला 


In [150]:
import tensorflow_text as text  # Import the package

In [152]:
k += 1

# Get user input
user_input = input("Enter a word to translate: ")

# Create a tokenizer based on your vocabulary
tokenizer = text.WhitespaceTokenizer()  # Or a suitable tokenizer from tensorflow_text

# Preprocess the input
tokens = tokenizer.tokenize(user_input).numpy().reshape(-1)  # Tokenize and reshape
indices = [input_token_index.get(word.decode('utf-8'), 0) for word in tokens]  # Convert to indices

# Convert to tensor
input_seq = tf.convert_to_tensor([indices])

# Decode the input
decoded_sentence = decode_sequence(input_seq)

# Print the results
print('Input English word:', user_input)
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Enter a word to translate: i am a boy
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
Input English word: i am a boy
Predicted Hindi Translation:  अवैध 
